In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# ==== STEP 1: Load and preprocess images ==== Harsh
dataset_path = r"C:\Users\kunjc\Downloads\gas"
image_size = (64, 64)
X = []
brands = []
sizes = []

print("Loading data...")
for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.isdir(folder_path):
        continue

    try:
        brand_label, size_label = folder_name.split("_")
    except ValueError:
        print("Skipping unknown label folder:", folder_name)
        continue

    for file in os.listdir(folder_path):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(folder_path, file)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, image_size)
                img = img / 255.0  # normalize
                X.append(img)
                brands.append(0 if brand_label.upper() == "HP" else 1)
                sizes.append({"2KG": 0, "5KG": 1, "15KG": 2}[size_label.upper()])
            except:
                print("Error loading:", img_path)

X = np.array(X)
y_brand = to_categorical(np.array(brands), num_classes=2)
y_size = to_categorical(np.array(sizes), num_classes=3)

print("Total images loaded:", len(X))

# ==== STEP 2: Train-Test Split ====
X_train, X_val, yb_train, yb_val, ys_train, ys_val = train_test_split(
    X, y_brand, y_size, test_size=0.2, random_state=42
)

# ==== STEP 3: Data Augmentation ====
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# ==== STEP 4: Build Improved CNN Model ====
input_layer = Input(shape=(64, 64, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.5)(x)

brand_output = Dense(2, activation='softmax', name='brand_output')(x)
size_output = Dense(3, activation='softmax', name='size_output')(x)

model = Model(inputs=input_layer, outputs=[brand_output, size_output])

model.compile(
    optimizer='adam',
    loss={'brand_output': 'categorical_crossentropy', 'size_output': 'categorical_crossentropy'},
    metrics={'brand_output': 'accuracy', 'size_output': 'accuracy'}
)

model.summary()

# ==== STEP 5: Custom Multi-output Generator ====
def multi_output_generator(datagen, X, yb, ys, batch_size):
    gen = datagen.flow(X, yb, batch_size=batch_size, shuffle=True, seed=42)
    while True:
        xb, yb_batch = next(gen)
        idxs = np.random.choice(len(ys), size=len(xb), replace=False)
        ys_batch = ys[idxs]
        yield xb, {'brand_output': yb_batch, 'size_output': ys_batch}

batch_size = 32
train_gen = multi_output_generator(datagen, X_train, yb_train, ys_train, batch_size)
steps_per_epoch = len(X_train) // batch_size

# ==== STEP 6: Callbacks ====
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("gas_cylinder_model.keras", save_best_only=True, monitor='val_loss', verbose=1)

# ==== STEP 7: Train Model ====
history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_val, {'brand_output': yb_val, 'size_output': ys_val}),
    epochs=100000,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

print("Best model saved as gas_cylinder_model.keras")

# ==== STEP 8: Evaluate Model ====
val_loss, val_brand_loss, val_size_loss, val_brand_acc, val_size_acc = model.evaluate(
    X_val, {'brand_output': yb_val, 'size_output': ys_val}, verbose=1
)

print(f"\nFinal Validation Accuracy:")
print(f"Brand Accuracy: {val_brand_acc * 100:.2f}%")
print(f"Size Accuracy : {val_size_acc * 100:.2f}%")

Loading data...
Total images loaded: 4702


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 64, 64, 32)        │             896 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 64, 64, 32)        │             128 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 32, 32, 32)        │               0 │ batch_normalization_3[0][… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 32, 32, 64)        │          18,496 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 32, 32, 64)        │             256 │ conv2d_4[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 16, 16, 64)        │               0 │ batch_normalization_4[0][… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 16, 16, 128)       │          73,856 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_5         │ (None, 16, 16, 128)       │             512 │ conv2d_5[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_5               │ (None, 8, 8, 128)         │               0 │ batch_normalization_5[0][… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 8192)              │               0 │ max_pooling2d_5[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 8192)              │               0 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ brand_output (Dense)          │ (None, 2)                 │          16,386 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ size_output (Dense)           │ (None, 3)                 │          24,579 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴───────────────

 Total params: 135,109 (527.77 KB)

 Trainable params: 134,661 (526.02 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/100000
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - brand_output_accuracy: 0.9601 - brand_output_loss: 0.2599 - loss: 1.9855 - size_output_accuracy: 0.5218 - size_output_loss: 1.7256
Epoch 1: val_loss improved from inf to 19.18067, saving model to gas_cylinder_model.keras
117/117 ━━━━━━━━━━━━━━━━━━━━ 59s 343ms/step - brand_output_accuracy: 0.9604 - brand_output_loss: 0.2586 - loss: 1.9827 - size_output_accuracy: 0.5219 - size_output_loss: 1.7240 - val_brand_output_accuracy: 0.0064 - val_brand_output_loss: 12.8145 - val_loss: 19.1807 - val_size_output_accuracy: 0.0021 - val_size_output_loss: 6.3767
Epoch 2/100000
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - brand_output_accuracy: 0.9939 - brand_output_loss: 0.0220 - loss: 1.4968 - size_output_accuracy: 0.5246 - size_output_loss: 1.4717
Epoch 2: val_loss improved from 19.18067 to 4.49287, saving model to gas_cylinder_model.keras
117/117 ━━━━━━━━━━━━━━━━━━━━ 38s 323ms/step - brand_output_accuracy: 0.9939 - brand_output_loss: 0.022

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from PIL import Image

# 1. Create dataframe from folder structure
base_path = r'C:\Users\kunjc\Downloads\gas'

data = []
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        brand, size = folder.split('_')
        for file in os.listdir(folder_path):
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                data.append({
                    'filepath': os.path.join(folder_path, file),
                    'brand': brand.upper(),
                    'size': size.upper()
                })

df = pd.DataFrame(data)

# 2. Label encoding
df['brand'] = df['brand'].map({'HP': 0, 'INDANE': 1})
df['size'] = df['size'].map({'2KG': 0, '5KG': 1, '15KG': 2})

# 3. Filter out classes with fewer than 2 samples
counts = df.groupby(['brand', 'size']).size().reset_index(name='count')
valid_combos = counts[counts['count'] >= 2][['brand', 'size']]
df_filtered = df.merge(valid_combos, on=['brand', 'size'])

# 4. Train-validation split (with stratify)
train_df, val_df = train_test_split(
    df_filtered,
    test_size=0.2,
    random_state=42,
    stratify=df_filtered[['brand', 'size']]
)

# 5. ImageDataGenerator
img_size = (128, 128)
batch_size = 32

def create_generator(df, datagen):
    return datagen.flow_from_dataframe(
        df,
        x_col='filepath',
        y_col=['brand', 'size'],
        target_size=img_size,
        class_mode='multi_output',
        batch_size=batch_size,
        shuffle=True
    )

# Keras doesn't support multi-output natively in flow_from_dataframe, so use custom generator
class MultiOutputDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, datagen, batch_size, img_size, shuffle=True):
        self.df = df.reset_index(drop=True)
        self.datagen = datagen
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = np.zeros((len(batch_df), *self.img_size, 3))
        brand_labels = np.zeros((len(batch_df), 1))
        size_labels = np.zeros((len(batch_df), 1))

        for i, row in enumerate(batch_df.itertuples()):
            img = Image.open(row.filepath).resize(self.img_size).convert('RGB')
            img = np.array(img) / 255.0
            X[i] = img
            brand_labels[i] = row.brand
            size_labels[i] = row.size

        return X, {'brand_output': brand_labels, 'size_output': size_labels}

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

datagen = ImageDataGenerator()

train_gen = MultiOutputDataGenerator(train_df, datagen, batch_size, img_size)
val_gen = MultiOutputDataGenerator(val_df, datagen, batch_size, img_size, shuffle=False)

# 6. Model architecture (Multi-output CNN)
input_layer = Input(shape=(*img_size, 3))
x = Conv2D(32, (3,3), activation='relu')(input_layer)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)

# Branch 1: Brand (HP vs INDANE)
brand_output = Dense(1, activation='sigmoid', name='brand_output')(x)

# Branch 2: Size (2KG, 5KG, 15KG)
size_output = Dense(3, activation='softmax', name='size_output')(x)

model = Model(inputs=input_layer, outputs=[brand_output, size_output])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'brand_output': 'binary_crossentropy',
        'size_output': 'sparse_categorical_crossentropy'
    },
    metrics={
        'brand_output': 'accuracy',
        'size_output': 'accuracy'
    }
)

model.summary()

# 7. Train model
callbacks = [EarlyStopping(patience=5, restore_best_weights=True)]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=callbacks
)

# 8. Save the model
model.save("gas_cylinder_model.keras")  # Modern format



Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_17 (Conv2D)            │ (None, 126, 126, 32)      │             896 │ input_layer_6[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_17              │ (None, 63, 63, 32)        │               0 │ conv2d_17[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_18 (Conv2D)            │ (None, 61, 61, 64)        │          18,496 │ max_pooling2d_17[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_18              │ (None, 30, 30, 64)        │               0 │ conv2d_18[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_6 (Flatten)           │ (None, 57600)             │               0 │ max_pooling2d_18[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_6 (Dropout)           │ (None, 57600)             │               0 │ flatten_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ brand_output (Dense)          │ (None, 1)                 │          57,601 │ dropout_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ size_output (Dense)           │ (None, 3)                 │         172,803 │ dropout_6[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 249,796 (975.77 KB)

 Trainable params: 249,796 (975.77 KB)

 Non-trainable params: 0 (0.00 B)

C:\Users\kunjc\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 350s 3s/step - brand_output_accuracy: 0.9700 - brand_output_loss: 0.1010 - loss: 0.2467 - size_output_accuracy: 0.9489 - size_output_loss: 0.1446 - val_brand_output_accuracy: 0.9947 - val_brand_output_loss: 0.0123 - val_loss: 0.0180 - val_size_output_accuracy: 0.9979 - val_size_output_loss: 0.0054
Epoch 2/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 377s 3s/step - brand_output_accuracy: 0.9959 - brand_output_loss: 0.0159 - loss: 0.0190 - size_output_accuracy: 0.9992 - size_output_loss: 0.0031 - val_brand_output_accuracy: 0.9968 - val_brand_output_loss: 0.0070 - val_loss: 0.0118 - val_size_output_accuracy: 0.9979 - val_size_output_loss: 0.0046
Epoch 3/30
118/118 ━━━━━━━━━━━━━━━━━━━━ 321s 3s/step - brand_output_accuracy: 0.9978 - brand_output_loss: 0.0063 - loss: 0.0092 - size_output_accuracy: 0.9998 - size_output_loss: 0.0028 - val_brand_output_accuracy: 0.9957 - val_brand_output_loss: 0.0070 - val_loss: 0.0126 - val_size_output_accuracy: 0.9979 - val_size_out

In [9]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tqdm import tqdm

# === Paths ===
test_path = r'C:\Users\kunjc\Downloads\gas_test'  # Folder with test images
model_path = 'gas_cylinder_model.keras'
img_size = (128, 128)

# === Label maps (for prediction output) ===
inv_brand_map = {0: 'HP', 1: 'INDANE'}
inv_size_map = {0: '2KG', 1: '5KG', 2: '15KG'}

# === Load model ===
model = load_model(model_path)

# === Process and Predict ===
results = []

for file in tqdm(os.listdir(test_path)):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        try:
            img_path = os.path.join(test_path, file)
            img = Image.open(img_path).resize(img_size).convert('RGB')
            img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

            brand_prob, size_prob = model.predict(img_array, verbose=0)
            brand_pred = int(brand_prob[0] > 0.5)
            size_pred = np.argmax(size_prob[0])

            results.append({
                'filename': file,
                'predicted_brand': inv_brand_map[brand_pred],
                'predicted_size': inv_size_map[size_pred]
            })

        except Exception as e:
            print(f"Error processing {file}: {e}")

# === Show predictions ===
import pandas as pd
df_results = pd.DataFrame(results)
print(df_results)

# === Optional: Save to CSV ===
df_results.to_csv("predictions_only.csv", index=False)
print("\n✅ Predictions saved to 'predictions_only.csv'")


  0%|                                                                                           | 0/12 [00:00<?, ?it/s]C:\Users\kunjc\AppData\Local\Temp\ipykernel_19164\264425509.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  brand_pred = int(brand_prob[0] > 0.5)
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  8.05it/s]

      filename predicted_brand predicted_size
0    HP2.1.jpg              HP            2KG
1    HP2.2.jpg              HP            2KG
2      HP2.jpg              HP            2KG
3    HP5.1.jpg              HP            5KG
4    HP5.2.jpg              HP            5KG
5      HP5.jpg              HP            5KG
6   ID15.1.jpg          INDANE           15KG
7   ID15.2.jpg          INDANE           15KG
8     ID15.jpg          INDANE           15KG
9    ID5.1.jpg          INDANE            5KG
10   ID5.2.jpg          INDANE            5KG
11     ID5.jpg          INDANE            5KG

✅ Predictions saved to 'predictions_only.csv'


In [7]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_17 (Conv2D)            │ (None, 126, 126, 32)      │             896 │ input_layer_6[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_17              │ (None, 63, 63, 32)        │               0 │ conv2d_17[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_18 (Conv2D)            │ (None, 61, 61, 64)        │          18,496 │ max_pooling2d_17[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_18              │ (None, 30, 30, 64)        │               0 │ conv2d_18[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_6 (Flatten)           │ (None, 57600)             │               0 │ max_pooling2d_18[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_6 (Dropout)           │ (None, 57600)             │               0 │ flatten_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ brand_output (Dense)          │ (None, 1)                 │          57,601 │ dropout_6[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ size_output (Dense)           │ (None, 3)                 │         172,803 │ dropout_6[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 749,390 (2.86 MB)

 Trainable params: 249,796 (975.77 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 499,594 (1.91 MB)

In [ ]:
venv\Scripts\activate
python app.py
http://127.0.0.1:5000